# Récupération de l'expérience des entrepreneurs; de la concurrence et de la demande par secteurs d'activités et localités; des latitudes et longitudes :

### I - Récupération des entreprises crées en IDF depuis 2010 jusqu'en 2024 pour les secteurs du commerce, de la restauration et de l'hôtellerie ; Récupération des expériences passés des entrepreneurs : 

Le but de cette partie est d'obtenir un data frame sur lequel nous travaillerons tout au long de ce projet. Nous décidons de récupérer l'ensemble des entreprises françaises crées entre 2010 et 2024, dont l'adresse principale est en Ile de France, pour les secteurs du commerce, de l'hôtellerie et la restauration. 

Notre choix s'explique par le fait que récupérer des entreprises crées à des dates trop lointaines pourraient fausser les résultats de la modélisation choisie, puisque par exemple, le tissu économique français des années 80 est drastiquement différent de celui d'aujourd'hui. 
De même, récupérer trop de secteurs différents pourrait entraîner une incapacité du modèle que l'on choisira à discriminer entre les entreprises radiées et non-radiées, qui sera notre variable cible à prédire.

Le choix de l'Ile de France s'explique aussi pour les mêmes raisons.

Ensuite, nous récupérons des indicateurs portants sur le nombre de concurrents par secteurs et par localités.

Enfin, nous récupérons des indicateurs sur les demandes des ménages par secteurs et par localités.


--------------------
Nous donnons ici une suite de fonction permettant de récupérer ce data frame. Nous récupérons au passage des variables concernant l'expérience potentielle des associés et des personnes physiques, qui nécessitent un traitement assez lourd. Pour simplifier les choses, on considéra qu'une personne physique est l'associé 0.

Voici la stratégie retenue pour n'obtenir qu'un seul data frame exploitable.

- La fonction `get_RNE_Exp` permet de récupérer l'expérience des associés. Cette fonction est la raison principale pour laquelle nous incluons cette partie de notre travail dans la partie récupération des données, et non dans celle de l'analyse et feature engineering.
En effet, nous voulons pour chaque entreprise d'IDF crées de 2010 à 2024 dans les secteurs visés, compter le nombre de fois où chaque associé à été enrôlé ou non dans une entreprise passé, c'est-à-dire crées avant la date de création concernant l'entreprise que nous regardons. Nous voulons aussi n'effectuer ce comptage que si l'associé à eu un rôle pertinent dans l'entreprise, le rendant le plus susceptible d'être à l'origine de la création bien que cela ne soit pas garanti. Enfin, nous voulons également compter le nombre d'entreprise radiée parmi les entreprises dans lesquelles l'associé en question à été enrôlé. Nous pensons en effet que ces variables permettront d'enrichir en information notre base de données, donnant plus de chance à un modèle de machine learning d'aboutir à des métriques satisfaisantes.
Pour effectuer cela, nous voulons travailler avec deux data frames : l'un dit de "travaille", étant celui sur lequel nous nous concentrons (idf_10_24, appelé simplement df_p dans la fonction), l'autre dit "historique" (total_p), comprenant l'ensemble des entreprises crées partout en France, pour les "grands" secteurs du commerce, artisanat et artisanat réglementé, depuis 1980 jusqu'en 2024. Pour chaque entreprise dans idf_10_24, nous regardons si le combo nom + prénom + date de naissance pour un associé donné correspond à une entreprise crées dans le passé. Si oui, nous sélectionnons seulement les entreprises dans lesquels l'associé à eu un rôle important. Nous récupérons au passage le nombre d'entreprises passées radiées. 
Comme cette tâche nécessite de travailler avec un data frame de plus de 8 000 000 de lignes (total), pandas n'est pas adapté ici. Nous utilisons alors DuckDB permettant de faire des requêtes SQL simple et rapide. 
`On récupère ainsi les variables total_expi (expérience total de l'associé i), total_radi (nombre d'entreprises passés radiées liées à l'associé i) et ape_truei (nombre d'entreprises passés ayant un lien avec le secteur d'activité de l'entreprise actuelle liée à l'associé i).`


Remarque : l'id (nom + prénom + date de naissance) est ici utilisé par défaut puisque les variables "associe_i_id" ne trouvent pas de correspondance dans le passé. Elles sont utiles simplement à l'identification interne d'un associé dans une entreprise donné.
D'autre part, il se peut que deux personnes différentes partagent cet id, même si cela est rare. Ceci sera un point d'amélioration à considérer.

- La fonction `get_RNE_MeanAge` permet de récupérer l'âge moyen des associées ainsi que le nombre d'associé.


- La fonction `get_RNE_Final` permet de récupérer les données depuis le cloud ssp cloud S3 puis les préparer pour la fonction `get_RNE_Exp` ainsi pouvoir récupérer l'expérience de chaque associé. Elle récupère ensuite l'âge moyen des associés et le nombre d'associé grâce à la fonction `get_RNE_MeanAge`. Elle renvoie un data frame pandas. Cette fonction se veux flexible pour que les lecteurs de ce projet puissent récupérer d'autres années, départements, activités...

- Finalement, nous exportons au format parquet le data frame obtenu sur le cloud, qui servira à la deuxième partie de la récupération de données.
Voire plus loin pour le lien de téléchargement de ce data frame.

In [ ]:
#Packages utilisés
import json
import pandas as pd
import zipfile
from tqdm import tqdm
import datetime
from ftplib import FTP
import getpass
import os
import s3fs
import numpy as np
import duckdb

In [ ]:

def get_RNE_Exp(numero_associe : int, df_p : pd.DataFrame, total_p : pd.DataFrame) -> pd.DataFrame:

    """
    Fonction qui permet de récupérer l'expérience d'un associée donné dont le numéro est compris en 1 et 5 ou bien celle
    d'une personne physique (0).

    
    inputs : 
            - numero_associe : int compris en entre 0 et 5 (0 == personne physique)
            - df_p : le dataframe RNE pour lequel on veut obtenir l'expérience des associés/personnes physiques
            - total_p : le dataframe RNE complet servant d'historique

    outputs :
            - res : dataframe avec 4 colonnes : siren, total_expi, total_radi, ape_truei, dont siren corresponds aux siren de 
                    df.
    
    total_exp0, total_rad0 et ape_true0 concernent les personnes physiques



    """
    if numero_associe > 0:
        ass1_nom = f"associe_{numero_associe}_nom"
        ass1_prenom = f"associe_{numero_associe}_prenom"
        ass1_naissance = f"associe_{numero_associe}_naissance"
    else:
        ass1_nom = "nom"
        ass1_prenom = "prenoms"
        ass1_naissance = "age_physique"

    df = df_p[["siren","date_creation",ass1_nom,ass1_prenom,ass1_naissance,"associe_1_role","codeAPE"]].copy()

    total = (total_p[["siren","date_creation",
    "associe_1_nom","associe_1_prenom","associe_1_naissance","associe_1_role",
    "associe_2_nom","associe_2_prenom","associe_2_naissance","associe_2_role",
    "associe_3_nom","associe_3_prenom","associe_3_naissance","associe_3_role",
    "associe_4_nom","associe_4_prenom","associe_4_naissance","associe_4_role",
    "associe_5_nom","associe_5_prenom","associe_5_naissance","associe_5_role",
    "nom","prenoms","age_physique","dateRadiation","codeAPE"]]).copy()

    df[ass1_nom] = df[ass1_nom].str.lower()
    df[ass1_prenom] = df[ass1_prenom].str.lower()
    df[ass1_naissance] = df[ass1_naissance].str.lower()


    associe_nom = [f"associe_{i}_nom" for i in range(1,6)]
    associe_prenom = [f"associe_{i}_prenom" for i in range(1,6)]
    associe_naissance = [f"associe_{i}_naissance" for i in range(1,6)]
    for i in range(len(associe_nom)):
        total[associe_nom[i]] = total[associe_nom[i]].str.lower()
        total[associe_prenom[i]] = total[associe_prenom[i]].str.lower()
        total[associe_naissance[i]] = total[associe_naissance[i]].str.lower()

    total["nom"] = total["nom"].str.lower()
    total["prenoms"] = total["prenoms"].str.lower()
    total["age_physique"] = total["age_physique"].str.lower()

    df["siren"] = df["siren"].astype(int)
    total["siren"] = total["siren"].astype(int)

    total['dateRadiation'] = pd.to_datetime(total['dateRadiation'], errors='coerce')
    

    #Connexion DuckDB
    con = duckdb.connect()

    #Enregistrement des DataFrames pandas dans DuckDB
    con.register("df", df)        
    con.register("total", total)

    con.execute("""
    CREATE OR REPLACE VIEW total_long AS
    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_1_nom AS sname,
        associe_1_prenom AS fname,
        associe_1_naissance AS birth,
        associe_1_role AS role
    FROM total
    WHERE associe_1_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_1_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_1_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_1_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_2_nom,
        associe_2_prenom,
        associe_2_naissance,
        associe_2_role
    FROM total
    WHERE associe_2_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_2_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_2_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_2_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_3_nom,
        associe_3_prenom,
        associe_3_naissance,
        associe_3_role
    FROM total
    WHERE associe_3_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_3_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_3_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_3_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_4_nom,
        associe_4_prenom,
        associe_4_naissance,
        associe_4_role
    FROM total
    WHERE associe_4_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_4_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_4_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_4_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_5_nom,
        associe_5_prenom,
        associe_5_naissance,
        associe_5_role
    FROM total
    WHERE associe_5_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_5_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_5_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_5_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        nom,
        prenoms,
        age_physique,
        associe_5_role
    FROM total
    WHERE nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND prenoms NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND age_physique NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique');
    """)

    if numero_associe > 0:
        #Requête principale pour l'historique d'un associé
        res = con.execute(f"""
        SELECT
            df.siren,


            COUNT(DISTINCT t.siren) AS total_exp{numero_associe},


            COUNT(DISTINCT CASE
                WHEN t.dateRadiation IS NOT NULL THEN t.siren
            END) AS total_rad{numero_associe},

            COUNT(DISTINCT CASE
                WHEN SUBSTR(t.codeAPE, 1, 1) = SUBSTR(df.codeAPE, 1, 1) THEN t.siren
            END) AS ape_true{numero_associe}

        FROM df

        LEFT JOIN total_long t
        ON df.associe_{numero_associe}_nom = t.sname
        AND df.associe_{numero_associe}_prenom = t.fname
        AND df.associe_{numero_associe}_naissance = t.birth
        AND t.date_creation < df.date_creation

        GROUP BY df.siren
        ORDER BY df.siren
        """).df()


    else:
        #Requête principale pour  l'historique d'une personne physique
        res = con.execute(f"""
        SELECT
            df.siren,


            COUNT(DISTINCT t.siren) AS total_exp{numero_associe},


            COUNT(DISTINCT CASE
                WHEN t.dateRadiation IS NOT NULL THEN t.siren
            END) AS total_rad{numero_associe},

            COUNT(DISTINCT CASE
                WHEN SUBSTR(t.codeAPE, 1, 1) = SUBSTR(df.codeAPE, 1, 1) THEN t.siren
            END) AS ape_true{numero_associe}

        FROM df

        LEFT JOIN total_long t
        ON df.nom = t.sname
        AND df.prenoms = t.fname
        AND df.age_physique = t.birth
        AND t.date_creation < df.date_creation

        GROUP BY df.siren
        ORDER BY df.siren
        """).df()

    return res



In [ ]:
def get_RNE_MeanAge(df : pd.DataFrame) -> pd.DataFrame:

    """
    Fonction permettant de récupérer l'âge moyen des associés ainsi que le nombre d'associé pour chaque entreprise données.

    inputs : 
            - df : data frame RNE
    
    ouputs : 
            - df : data frame enrichi de l'âge moyen des associés ainsi que le nombre d'associé.
    """
    
    #Récupération du nombre d'associés
    df["is1"] = (~df["associe_1_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["is2"] = (~df["associe_2_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["is3"] = (~df["associe_3_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["is4"] = (~df["associe_4_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["is5"] = (~df["associe_5_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["personneMorale"] = df["personneMorale"].astype(int)

    df["nb_associe"] = df["is1"] + df["is2"]  + df["is3"] + df["is4"] + df["is5"] + (1-df["personneMorale"])
    
    #Récupération de l'âge moyen des associés à la création d'entreprises
    df["associe_1_naissance"] = (df["associe_1_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_1_naissance"] = pd.to_datetime(df["associe_1_naissance"])

    df["associe_2_naissance"] = (df["associe_2_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_2_naissance"] = pd.to_datetime(df["associe_2_naissance"])

    """
    #Corrige un bug natif
    i_error = df.loc[df["associe_3_naissance"] == "0997-08"]["associe_3_naissance"].index
    df[i_error, "associe_3_naissance"] = "1997-08"
    """

    df["associe_3_naissance"] = (df["associe_3_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_3_naissance"] = pd.to_datetime(df["associe_3_naissance"],errors='coerce')


    df["associe_4_naissance"] = (df["associe_4_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_4_naissance"] = pd.to_datetime(df["associe_4_naissance"])

    df["associe_5_naissance"] = (df["associe_5_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_5_naissance"] = pd.to_datetime(df["associe_5_naissance"])

    df["age_physique"] = (df["age_physique"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["age_physique"] = pd.to_datetime(df["age_physique"])
 
    
    df["age0"] = (df["date_creation"] - df["age_physique"])
    df["age1"] = (df["date_creation"] - df["associe_1_naissance"])
    df["age2"] = (df["date_creation"] - df["associe_2_naissance"])
    df["age3"] = (df["date_creation"] - df["associe_3_naissance"])
    df["age4"] = (df["date_creation"] - df["associe_4_naissance"])
    df["age5"] = (df["date_creation"] - df["associe_5_naissance"])

   
    df["age0"] = df["age0"].dt.total_seconds() / (86400 * 365.25)
    df["age1"] = df["age1"].dt.total_seconds() / (86400 * 365.25)
    df["age2"] = df["age2"].dt.total_seconds() / (86400 * 365.25)
    df["age3"] = df["age3"].dt.total_seconds() / (86400 * 365.25)
    df["age4"] = df["age4"].dt.total_seconds() / (86400 * 365.25)
    df["age5"] = df["age5"].dt.total_seconds() / (86400 * 365.25)


    df["mean_age"] = (
        df[["age0","age1", "age2", "age3", "age4", "age5"]]
        .sum(axis=1, skipna=True)
        / (df["nb_associe"])
    )

    return df

### II - Récupération du nombre de concurrent à une date donné par secteur et localité et récupération du nombre de personnes par commune : 

In [ ]:
def adapt_date(year : int) -> int:
    """
    A commenter
    """

    if year == 2010:
        res = 2011
    elif year > 2022:
        res = 2022
    else :
        res = year 
    return res



def get_RNE_Pop(df_p : pd.DataFrame,df_pop : pd.DataFrame) -> pd.DataFrame : 

    """
    A commenter
    """

    df_pop = df_pop.loc[df_pop["RP_MEASURE"] == "DWELLINGS_POPSIZE"][["GEO","TIME_PERIOD","OBS_VALUE"]]

    #renommage des noms des colonnes d'intérêts pour matching avec df_p
    df_pop = df_pop.rename(columns={"GEO": "codeInseeCommune"})
    df_pop = df_pop.rename(columns={"TIME_PERIOD": "annee_tronc"})
    df_pop = df_pop.rename(columns={"OBS_VALUE": "pop_commune"})

    #Conversion en str par sécurité
    df_pop["codeInseeCommune"] = df_pop["codeInseeCommune"].astype(str)
    df_p["codeInseeCommune"] = df_p["codeInseeCommune"].astype(str)

    df_p["annee_tronc"] = df_p["date_creation"].dt.year
    df_p["annee_tronc"] = df_p["annee_tronc"].astype(int)

    #Adaptation des dates : approximation de 2010 par 2011 et > 2022 par 2022 (voire adapt_date)
    df_p["annee_tronc"] = df_p["annee_tronc"].apply(adapt_date)
    
    df_p = df_p.merge(df_pop[["codeInseeCommune","annee_tronc","pop_commune"]], 
              on=["codeInseeCommune","annee_tronc"], 
              how='left')
    
    #On remplace par la dernière valeur connue pour une commune donné si année absente dans df_pop
    df_p["pop_commune"] = df_p.groupby("codeInseeCommune")["pop_commune"].ffill()
    
    return df_p


In [1]:
def get_RNE_local_concurrents(df_p : pd.DataFrame, total_p : pd.DataFrame) -> pd.DataFrame :

      #Récupération de l'indicatrice 1 si radiée 0 sinon
      total_p["radié"] = total_p["dateRadiation"].apply(lambda x : 1 if x != None else 0)

      #Conversion en date time de date_creation par sécurité
      total_p["date_creation"] = pd.to_datetime(total_p["date_creation"])
      df_p["date_creation"] = pd.to_datetime(df_p["date_creation"])
      

      total = total_p.copy()
      df = df_p.copy()

      #Connexion DuckDB
      con = duckdb.connect()

      #Enregistrement des DataFrames pandas dans DuckDB
      con.register("df", df)        
      con.register("total", total)

      con.execute("""
      CREATE OR REPLACE VIEW total_long AS
      SELECT siren, date_creation,
            radié AS rad,
            codeAPE as ape,
            code_postal as code
      FROM total
      WHERE radié IS NOT NULL;
      """)

      #Requête principale pour le nombre de concurrents encore en poste sur le marché local
      res = con.execute(f"""
      SELECT
      df.siren,

      COUNT(DISTINCT t.siren) AS nb_local_concurrents

      FROM df

      LEFT JOIN total_long t
      ON df.codeAPE = t.ape
      AND df.code_postal = t.code
      AND t.date_creation <= df.date_creation

      GROUP BY df.siren
      ORDER BY df.siren
      """).df()

      con.close()

      return res
    

NameError: name 'pd' is not defined

In [ ]:
#Pour récupérer le nombre de personnes par communes
url = "https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_supp/DS_RP_SERIE_HISTORIQUE_2022_data.csv"

df_pop = pd.read_csv(url,sep=";")

In [ ]:
def get_RNE_Final(dep : list, activites : list, date_min : list)-> pd.DataFrame:

    """
    Fonction permettant de récupérer les données du RNE sur le cloud ssp cloud S3, de récupérer les expériences passés des 
    associés, puis l'âge moyen des associés, ainsi que leur nombre, cela pour une liste de département et d'activités données,
    ainsi qu'une date minimale de création.

    inputs :
            - dep : liste de département de type ["78","01",...]
            - activites : liste d'activités par "bribes" de code APE de type ["45","5610B",...]
            - date_min : date minimale de création de type [2021,1,1] == [y,month,day]
    
    outputs : 
            - df_return : data frame RNE
    """


    ##################################################################################################
    #    ETAPE 1 : Récupération des données selon dep, acitivtes et date_min + l'historique total    #
    ##################################################################################################


    #Récupération des données depuis le cloud ssp cloud S3
    dates_select = [1980,1990,2000,2005,2010,2015,2017,2018,2019,2020,2021,2022,2023,2024]

    dfs_rne = dict()
    df_names = [(f"df_{dates_select[i-1]}_{dates_select[i]}",i) for i in range(1,len(dates_select))]

    url_dataRNE = ['https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_1980_1990.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_1990_2000.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2000_2005.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2005_2010.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2010_2015.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2015_2017.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2017_2018.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2018_2019.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2019_2020.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2020_2021.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2021_2022.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2022_2023.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2023_2024.parquet']

    for names,i in tqdm(df_names):
        print(f"Récupération depuis le cloud {i}/13")
        url = url_dataRNE[i-1]
        df = pd.read_parquet(url)
        dfs_rne[names] = df
    #------------------------------------------------------------------------------------------------#

    #Récupération des colonnes et création d'un nouveau data frame vide
    columns_rne = list(dfs_rne[df_names[0][0]].columns)
    idf_10_24 = pd.DataFrame(columns=columns_rne)

    #Filtrage par activites et département
    for names in tqdm(df_names):
        print("Filtrage par département et activités")
        data = dfs_rne[names[0]]
        data = data[data["codeAPE"].str.startswith(tuple(activites)) & data["code_postal"].str.startswith(tuple(dep))]
        idf_10_24 = pd.concat([idf_10_24,data])
        
    #Filtrage pour les dates de création après 2010-01-01 (inlcus)
    idf_10_24["date_creation"] = pd.to_datetime(idf_10_24["date_creation"])

    print(f"Récupération depuis {date_min[0]}-{date_min[1]}-{date_min[2]}")
    idf_10_24 = idf_10_24[idf_10_24["date_creation"] >= datetime.datetime(date_min[0],date_min[1],date_min[2])]

    idf_10_24["dateRadiation"] = idf_10_24["dateRadiation"].apply(lambda x : 
        x if x not in ["non-disp","None"] else pd.NaT)


    #------------------------------------------------------------------------------------------------#

    #Récupération des colonnes et création d'un nouveau data frame vide
    columns_rne = list(dfs_rne[df_names[0][0]].columns)
    total = pd.DataFrame(columns=columns_rne)

    #Récupération complète dans un data frame
    print("Récupération de l'historique total")
    for names in tqdm(df_names):
        data = dfs_rne[names[0]]
        total = pd.concat([total,data])
    

    total["date_creation"] = pd.to_datetime(total["date_creation"])
    total["dateRadiation"] = total["dateRadiation"].apply(lambda x : 
        x if x not in ["non-disp","None"] else pd.NaT)

    #########################################################################################################
    #   ETAPE 2 : Récupération de l'expérience des associés/personnes physiques + age moyen à la création   #
    #########################################################################################################


    
    #Récupération de l'expérience des associés/personnes physiques
    associe_list = [0,1,2,3,4,5]

    idf_10_24["siren"] = idf_10_24["siren"].astype(int)

    df_return = idf_10_24.copy()

    for i in associe_list:
        
        print(f"Récupération expérience associé {i}")
        res = get_RNE_Exp(numero_associe=i, df_p=idf_10_24, total_p=total)
        res["siren"] = res["siren"].astype(int)
        df_return = df_return.merge(res, on="siren", how="left")
        
    #Récupération de l'âge moyen à la création

    print("Récupération de l'âge moyen")
    df_return = get_RNE_MeanAge(df_return)
    

    return df_return
    

In [ ]:
#Listes des activités retenues par bribes d'APE (voire nomenclature NAF)
activites_annotees = [
    ("4511Z", "Commerce de voitures et véhicules légers"),
    ("4519Z", "Commerce d'autres véhicules automobiles"),
    ("45",    "Entretien et réparation véhicules automobiles"),
    ("454",   "Commerce et réparation de motocyclettes"),
    ("461",   "Intermédiaires du commerce de gros"),
    ("462",   "Commerce de gros produits agricoles et animaux vivants"),
    ("463",   "Commerce de gros aliments, boissons et tabac"),
    ("466",   "Commerce de gros autres équipements industriels"),
    ("467",   "Autres commerces de gros spécialisés"),
    ("471",   "Commerce de détail en magasin non spécialisé"),
    ("472",   "Commerce de détail alimentaire en magasin spécialisé"),
    ("473",   "Commerce de détail de carburants en magasin spécialisé"),
    ("478",   "Commerce de détail sur éventaires et marchés"),
    ("551",   "Hôtels et hébergement similaire"),
    ("552",   "Hébergement touristique et autre hébergement courte durée"),
    ("5610A", "Restauration traditionnelle"),
    ("5610B", "Cafétérias et autres libres-services"),
    ("5610C", "Restauration rapide"),
    ("563",    "Débits de boissons")
]

activites = [activites_annotees[i][0] for i in range(len(activites_annotees))]

#Départements d'IDF
dep = ["75","77","78","91","92","93","94","95"]

#Date minimale 
date_min = [2010,1,1]

data = get_RNE_Final(dep=dep, activites=activites,date_min=date_min)

In [ ]:
MY_BUCKET = "guillaume176"

FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/data_final/idf_10_24.parquet"

with fs.open(FILE_PATH_OUT_S3,"wb") as file_out:
    data.to_parquet(file_out, index=False)


Lien pour le téléchargement pour `idf_10_24`: 

In [ ]:
path_data = "guillaume176/diffusion/data_final"
file_list = fs.ls(path_data)[1:]
file_list = ["https://minio.lab.sspcloud.fr/" + f for f in file_list]
print(file_list[0])